In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
import os
import random

import numpy as np
import PIL.Image
from zeus.plotting.utils import axes

from kidney.datasets.utils import read_segmentation_info

In [ ]:
img_info = read_segmentation_info("/mnt/fast/data/cuts/mask_t10_256", file_format="enum")

In [ ]:
len(img_info)

In [ ]:
cols, rows = 7, 7
ax = axes(subplots=(cols, rows), figsize=(18, 18))
samples = random.sample(img_info, k=cols*rows)
for i in range(rows):
    for j in range(cols):
        x = samples[i + j*cols]
        img = np.asarray(PIL.Image.open(x["img"]))
        seg = np.asarray(PIL.Image.open(x["seg"]))
        ax[i][j].imshow(PIL.Image.fromarray(img))
        ax[i][j].imshow(PIL.Image.fromarray(seg), alpha=0.2)
        ax[i][j].axis("off")

In [ ]:
reader = get_reader()

In [ ]:
keys = reader.get_keys(SampleType.Labeled)

In [ ]:
CONFIG = {
    "256px": {
        "reduce": 4,
        "tile_size": 256,
    },
    "1024px": {
        "reduce": 1,
        "tile_size": 1024
    }
}

config_name = "1024px"

REDUCE_MULT = CONFIG[config_name]["reduce"]
TILE_SIZE = CONFIG[config_name]["tile_size"]
MOD = REDUCE_MULT * TILE_SIZE

In [ ]:
sz = TILE_SIZE

In [ ]:
sample = reader.fetch_one(keys[0])

In [ ]:
crops = cut_sample(sample, TILE_SIZE)

In [ ]:
img, seg = sample["image"], sample["mask"]

In [ ]:
padded = pad_sample({"img": img, "seg": seg}, TILE_SIZE)

In [ ]:
padded["img"].shape, padded["seg"].shape

In [ ]:
output_dir = "/tmp/images"

os.makedirs(output_dir, exist_ok=True)

for key in keys:
    sample = reader.fetch_one(key)
    crops = cut_sample(sample, TILE_SIZE)
    img, seg = sample["image"], sample["mask"]
    
    

In [ ]:
# h, w = shape = img.shape[:2]
# pad0 = (MOD - h%MOD)%MOD
# pad1 = (MOD - w%MOD)%MOD

In [ ]:
# (MOD - (h%MOD))%MOD

In [ ]:
# padding = [[pad0//2, pad0 - pad0//2], [pad1//2, pad1 - pad1//2]]
# img = np.pad(img, padding + [[0, 0]], constant_values=0)
# seg = np.pad(seg, padding, constant_values=0)

In [ ]:
# new_size = img.shape[1]//REDUCE_MULT, img.shape[0]//REDUCE_MULT
# img = cv.resize(img, new_size, interpolation=cv.INTER_AREA)
# img = img.reshape(img.shape[0]//sz, sz, img.shape[1]//sz, sz, 3)
# img = img.transpose(0, 2, 1, 3, 4).reshape(-1, sz, sz, 3)

In [ ]:
# seg = cv.resize(seg, new_size, interpolation=cv.INTER_NEAREST)
# seg = seg.reshape(seg.shape[0]//sz, sz, seg.shape[1]//sz, sz)
# seg = seg.transpose(0, 2, 1, 3).reshape(-1, sz, sz)

In [ ]:
# img.shape, seg.shape

In [ ]:
t_saturation = 40 
t_pixels = 200 * sz // 256
output_dir = "/tmp/images"

sz = TILE_SIZE

os.makedirs(output_dir, exist_ok=True)

for key in keys:
    sample = reader.fetch_one(key)
    img, seg = sample["image"], sample["mask"]

    h, w = img.shape[:2]
    pad0 = (MOD - h%MOD)%MOD
    pad1 = (MOD - w%MOD)%MOD
    
    padding = [[pad0//2, pad0 - pad0//2], [pad1//2, pad1 - pad1//2]]
    img = np.pad(img, padding + [[0, 0]], constant_values=0)
    seg = np.pad(seg, padding, constant_values=0)
    
    new_size = img.shape[1]//REDUCE_MULT, img.shape[0]//REDUCE_MULT
    img = cv.resize(img, new_size, interpolation=cv.INTER_AREA)
    img = img.reshape(img.shape[0]//sz, sz, img.shape[1]//sz, sz, 3)
    img = img.transpose(0, 2, 1, 3, 4).reshape(-1, sz, sz, 3)
    
    seg = cv.resize(seg, new_size, interpolation=cv.INTER_NEAREST)
    seg = seg.reshape(seg.shape[0]//sz, sz, seg.shape[1]//sz, sz)
    seg = seg.transpose(0, 2, 1, 3).reshape(-1, sz, sz)

    mean, mean_sq = [], []
    
    for i, (image, mask) in enumerate(zip(img, seg)):
        hsv = cv.cvtColor(image, cv.COLOR_RGB2HSV)
        h, s, v = cv.split(hsv)

        if (s > t_saturation).sum() <= t_pixels:
            continue

        if image.sum() <= t_pixels:
            continue

        mean.append((image/255.).reshape(-1, 3).mean(0))
        mean_sq.append(((image/255.)**2).reshape(-1, 3).mean(0))
        
        img_file = os.path.join(output_dir, f"img.{key}_{i}.png")
        PIL.Image.fromarray(image).save(img_file, format="png")

        seg_file = os.path.join(output_dir, f"seg.{key}_{i}.png")
        PIL.Image.fromarray(mask*255).save(seg_file, format="png")
        
    break

In [ ]:
img_info = read_segmentation_info("/tmp/images", file_format="enum")

In [ ]:
cols, rows = 5, 5
ax = axes(subplots=(cols, rows), figsize=(20, 20))
samples = random.sample(img_info, k=cols*rows)
for i in range(rows):
    for j in range(cols):
        x = samples[i + j*cols]
        img = np.asarray(PIL.Image.open(x["img"]))
        seg = np.asarray(PIL.Image.open(x["seg"]))
        ax[i][j].imshow(PIL.Image.fromarray(img))
        ax[i][j].imshow(PIL.Image.fromarray(seg), alpha=0.2)
        ax[i][j].axis("off")